In [27]:
import numpy as np
import pygame
import sys
import os

Cargar la base de datos de la simulación realizada

In [30]:
with open('ammonia_tank_mL_25.0e-6_kg_per_s.txt', 'r') as file:
    data = file.readlines()

Tank_area = np.pi * (0.201 ** 2) / 4 # d_i = 0.201 m
Tank_volume = 6.75e-3 # m^3
max_height = Tank_volume / Tank_area  # m
liquid_heights = []
liquid_volumes = []
times = []
BOG_rates = []
QV_values = []
BL_values = []

data = data[1:]
for element in data:
    cleaned_line = element.strip().split(',')
    print(float(cleaned_line[1]))
    # Add the values to the respective lists
    times.append(float(cleaned_line[0]))
    liquid_heights.append(float(cleaned_line[1]) / Tank_area)  # Convert m^3 to height in m
    liquid_volumes.append(float(cleaned_line[1]))
    BOG_rates.append(float(cleaned_line[2]))
    QV_values.append(float(cleaned_line[4]))
    BL_values.append(float(cleaned_line[5]))

0.0018765000000000001
0.0018786864893320298
0.00188087327198059
0.0018830600207132963
0.0018852465101859921
0.001887432984348649
0.0018896194453991517
0.0018918058949210883
0.0018939923340576732
0.0018961787637061516
0.0018983651845988225
0.0019005515974314245
0.0019027380026673377
0.001904924400831654
0.0019071107924431804
0.001909297177879563
0.0019114835574977154
0.0019136699317032712
0.0019158563009372732
0.0019180426654445675
0.0019202290254660389
0.0019224153814268194
0.0019246017334988465
0.0019267880818982985
0.0019289744270380477
0.0019311607688005646
0.0019333471078106163
0.0019355334439089685
0.0019377197775133173
0.0019399061085479518
0.0019420924373107086
0.0019442787640134322
0.0019464650886496858
0.001948651411374743
0.001950837732400578
0.0019530240517942534
0.0019552103696166266
0.0019573966859638583
0.001959583000939378
0.0019617693146640105
0.001963955627209062
0.0019661419386223728
0.001968328248977977
0.001970514558336718
0.001972700866824522
0.00197488717438927
0.

In [3]:
# Inicializar pygame
pygame.init()

# Dimensiones de la ventana
WIDTH, HEIGHT = 400, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Tanque esquemático")

# Colores
GRAY = (180, 180, 180)
WHITE = (255, 255, 255)
BLUE = (173, 216, 230)
PINK = (255, 220, 220)
BLACK = (0, 0, 0)

# Loop principal
running = True
while running:
    screen.fill(WHITE)
    
    # --- DIBUJO DEL TANQUE ---
    # Tanque externo
    tank_rect = pygame.Rect(120, 100, 160, 400)
    pygame.draw.rect(screen, GRAY, tank_rect, border_radius=80)
    
    # Fase líquida (parte inferior)
    liquid_rect = pygame.Rect(140, 360, 120, 120)
    # pygame.draw.rect(screen, BLUE, liquid_rect, border_radius=60)
    pygame.draw.rect(
    screen,
    BLUE,
    rect=liquid_rect,
    border_top_left_radius=0,
    border_top_right_radius=0,
    border_bottom_left_radius=60,
    border_bottom_right_radius=60
)
    
    # Fase vapor (parte superior)
    vapor_rect = pygame.Rect(140, 120, 120, 240)
    pygame.draw.rect(screen, PINK, vapor_rect, border_radius=60)
    pygame.draw.rect(
    screen,
    PINK,
    rect=vapor_rect,
    border_top_left_radius=60,
    border_top_right_radius=60,
    border_bottom_left_radius=0,
    border_bottom_right_radius=0
)
    
    # Contorno interior del tanque
    inner_rect = pygame.Rect(140, 120, 120, 360)
    pygame.draw.rect(screen, BLACK, inner_rect, 2, border_radius=60)
    
    # Línea de separación líquido-vapor
    pygame.draw.line(screen, BLACK, (140, 360), (260, 360), 2)
    
    # # --- Válvula superior ---
    # pygame.draw.polygon(screen, BLACK, [(200, 80), (190, 100), (210, 100)])  # triángulo inferior
    # pygame.draw.polygon(screen, BLACK, [(200, 60), (190, 80), (210, 80)])    # triángulo superior
    # pygame.draw.line(screen, BLACK, (200, 100), (200, 120), 3)  # tubo
    
    # # --- Bomba lateral ---
    # pygame.draw.line(screen, BLACK, (260, 420), (320, 420), 4)  # tubería horizontal
    # pygame.draw.circle(screen, GRAY, (340, 420), 20)             # cuerpo bomba
    # pygame.draw.circle(screen, BLACK, (340, 420), 20, 2)
    # pygame.draw.circle(screen, WHITE, (340, 420), 10)
    
    # Actualizar pantalla
    pygame.display.flip()
    
    # Salir del programa
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()
sys.exit()


SystemExit: 

/Users/alonsoespinozacorrea/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [40]:
def meters_to_pixels(meters, scale=294.18):
    return int(meters * scale)

pygame.init()

# Ventana
WIDTH, HEIGHT = 400, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Animación de nivel de líquido")

# Colores
GRAY = (180, 180, 180)
WHITE = (255, 255, 255)
BLUE = (173, 216, 230)
PINK = (255, 220, 220)
BLACK = (0, 0, 0)

# Create a Font object
font = pygame.font.SysFont('Arial', 16) # Using a system font, size 16

# Geometría del tanque
x_tank, y_tank, w_tank, h_tank = 120, 100, (120 + 60), (120 + 63)

# Datos de simulación (ejemplo)
time = times                 # 10 s simulados
height_liquid = liquid_heights  # ejemplo: sube asintóticamente

# Configurar reloj para controlar velocidad
clock = pygame.time.Clock()
fps = 120

# Escalar alturas a píxeles dentro del tanque
liquid_min = y_tank + h_tank - 20   # fondo
liquid_max = y_tank + 80            # parte superior del líquido

def get_liquid_y(h_fraction):
    # Convierte la fracción (0 a 1) en coordenada vertical
    return liquid_min - h_fraction * (liquid_min - liquid_max)

running = True
frame_count = 0
# Create a directory to store frames if it doesn't exist
if not os.path.exists("frames"):
    os.makedirs("frames")

# Loop principal de animación
for i in range(len(time)):
    screen.fill(WHITE)

    liquid_fraction = (liquid_volumes[i] / Tank_volume) * 100
    vapor_fraction = 100 - liquid_fraction

    # Render the text
    text_surface = font.render(f'Tank filling at {times[i]/3600:.2f} hr', True, BLACK) # Text, antialias, color
    text_surface2 = font.render(rf'Flow rate: m_L = 25e-6 kg/s', True, BLACK) # Text, antialias, color

    # Textos para indicar el volumen de líquido y vapor
    text_surface3 = font.render(rf'Lf = {liquid_fraction:.2f}%', True, BLACK) # Text, antialias, color
    text_surface4 = font.render(rf'Vf = {vapor_fraction:.2f}%', True, BLACK) # Text, antialias, color

    # Get the rectangle of the text surface for positioning
    text_rect = text_surface.get_rect()
    text_rect.center = (10 + WIDTH // 2, 20) # Center the text

    text_rect2 = text_surface2.get_rect()
    text_rect2.center = (10 + WIDTH // 2, 40) # Center the text
    
    screen.blit(text_surface2, text_rect2) # Blit the text to the screen
    screen.blit(text_surface, text_rect) # Blit the text to the screen

    h_bottom = (y_tank + h_tank - 20)
    h_surface_simulation = meters_to_pixels(height_liquid[i])

    # --- Tanque ---
    pygame.draw.rect(
        screen, GRAY, (x_tank, y_tank, w_tank, h_tank),
    )

    # Rectángulo interior del tanque
    pygame.draw.rect(
        screen, BLACK, (x_tank+20, y_tank+20, w_tank-40, h_tank-40),
        2, border_top_left_radius=60, border_top_right_radius=60
    )

    # --- Líquido ---
    # h_frac = (height_liquid[i] - min(height_liquid)) / (max(height_liquid) - min(height_liquid))
    h_frac = (height_liquid[i]) / (np.max(height_liquid))  # Normalizar con max_height
    height_px = h_frac * (liquid_min - liquid_max)
    y_liquid = liquid_min - height_px
    
    liquid_rect = pygame.Rect(x_tank+20, y_liquid, w_tank-40, height_px)
    pygame.draw.rect(screen, BLUE, liquid_rect)

    # --- Vapor ---
    vapor_rect = pygame.Rect(x_tank+20, y_tank+20, w_tank-40, (y_liquid - (y_tank+20)))
    pygame.draw.rect(screen, PINK, vapor_rect)

    # --- Línea de separación ---
    pygame.draw.line(screen, BLACK, (x_tank+20, y_liquid), (x_tank+w_tank-20, y_liquid), 2)

    # Gráfico de los porcentajes de líquido y vapor
    text_rect3 = text_surface3.get_rect()
    text_rect3.center = (10 + WIDTH // 2, 240) # Center the text

    text_rect4 = text_surface4.get_rect()
    text_rect4.center = (10 + WIDTH // 2, 150) # Center the text

    screen.blit(text_surface4, text_rect4) # Blit the text to the screen
    screen.blit(text_surface3, text_rect3) # Blit the text to the screen

    # Guardar cada frame como imagen
     # Save the current frame
    frame_filename = os.path.join("frames", f"frame_{frame_count:04d}.png")
    pygame.image.save(screen, frame_filename)
    frame_count += 1


    # Actualizar pantalla
    pygame.display.flip()
    clock.tick(fps)

    # Permitir salir del programa
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

pygame.quit()

Create a gif from the animation

In [ ]:
    from PIL import Image
    import glob

    # Get a list of all saved frame images
    frame_files = sorted(glob.glob("frames/frame_*.png"))

    # Open the first image to use as the base for the GIF
    images = [Image.open(f) for f in frame_files]

    # Save as an animated GIF
    images[0].save(
        "animated_pygame.gif",
        save_all=True,
        append_images=images[1:],
        duration=100, # Duration for each frame in milliseconds
        loop=0 # 0 means infinite loop
    )

    print("GIF created successfully!")

GIF created successfully!


: 